We need a couple of known things for this setup to work

Initially that'll be a property that can be used to state what other properties map to

As well as a property that can be used to say Items and Properties map to Wikidata.

In [1]:
import json
import re
from wikidataintegrator import wdi_core, wdi_login

with open('./.secret.json') as f:
    data = json.load(f)
    wb_username = data['demo_user']
    wb_password = data['demo_password']
wb_endpoint = 'https://overture-demo.wikibase.cloud/w/api.php'
wb_login = wdi_login.WDLogin(user=wb_username, pwd=wb_password, mediawiki_api_url=wb_endpoint)

# Define properties that need to exist in a map
map = {
    'root-map': {
        "label": "Root map",
        "aliases": ["Maps to"],
        "description": "Entity maps to source",
        "datatype": "string",
    },
    'location': {
        "label": "Location",
        "aliases": ["Coordinate Location", "Coordinates"],
        "description": "Location of the entity",
        "datatype": "globe-coordinate",
    },
    'wikidata': {
        "label": "Wikidata",
        "aliases": ["Qid", "Wikidata ID"],
        "datatype": "string",
        "root-map": "wikidata"
    },
}

# Lookup to see if these properties already exist
existing_properties = wdi_core.WDItemEngine.get_wd_search_results(list(map.keys()), mediawiki_api_url=wb_endpoint)
for existing_property in existing_properties:
    # TODO add the ID to the list :D but for now die as this didn't occour yet...
    print("Property already exists: " + existing_property['label'])
    exit()

# # Create properties
for key, value in map.items():
    if 'id' in value:
        print("Property already exists: " + value['label'] + " is " + value['id'])
        continue
    property = wdi_core.WDItemEngine(mediawiki_api_url=wb_endpoint)
    property.set_label(value['label'])
    if 'aliases' in value:
        property.set_aliases(value['aliases'])
    if 'description' in value:
        property.set_description(value['description'])
    if 'root-map' in value:
        property.statements.append(wdi_core.WDString(value['root-map'], prop_nr=map['root-map']['id']))
    # Write and catch errors, as wikibase.cloud can be a bit slow in allowing us to lookup existing things sometimes
    try:
        property.write(login=wb_login,entity_type='property', property_datatype=value['datatype'])
        map[key]['id'] = property.wd_item_id
        print("Created property: " + value['label'] + " is " + property.wd_item_id)
    except Exception as e:
        if 'label-conflict' in str(e):
            # Add it to the map
            map[key]['id'] = re.search(r'\[\[Property:([^|]+)\|', str(e)).group(1)
            print("Property already exists: " + value['label'] + " is " + map[key]['id'])
        else:
            print("Error creating property: " + str(e))
            exit()

print("These proeprties should now all exist")

# Simplify the map
for key, value in map.items():
    map[key] = value['id']
# And write to a file
with open('./property_map.json', 'w') as f:
    json.dump(map, f, indent=2)

Error while writing to Wikidata
Property already exists: Root map is P1
Created property: Location is P28
Error while writing to Wikidata
Property already exists: Wikidata is P22
These proeprties should now all exist
